# Data engineering for NeuralProphet/LSTM (Store_chain_id = 2)

Overall Process:
- Extract and group transactions for store chain id = 2
- Remove outliers
- Check for NULL values 
- Create new features
    1. Weekend vs Weekday flag )
    2. Daily number of customers 
    3. Purchase amount/Purchase quantity
- Create separate dataset for z-score filtered data (for testing model performance)

Packages:
1. pandas
2. matplotlib
3. scipy

### 1. Extract transactions with store_chain_id = 2
- Total number of rows = 3,773,005

In [ ]:
-- Extract transaction data for store chain = 2
CREATE OR REPLACE TABLE store_2_preprocessed_transactions AS
SELECT * FROM preprocessed_transactions
WHERE store_chain_id = 2;

In [ ]:
SELECT * FROM store_2_preprocessed_transactions;

### 2. Remove outliers 
- Based on box plots, outliers are as followed:
    1. Purchase amount > 3000
    2. Purchase quantity > 400

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

session = get_active_session()
session.use_database("ml")
session.use_schema("retail_store")

transactions_df = session.table("store_2_preprocessed_transactions").select("purchase_amount", "purchase_quantity")
transactions_df = transactions_df.to_pandas()

- Purchase amount outlier

In [ ]:
# Purchase amount box plot
plt.figure(figsize =(3, 3))
plt.boxplot(transactions_df["PURCHASE_AMOUNT"])
plt.show()

- Remove purchase amount > 3000 and plot boxplot again.

In [ ]:
# Purchase amount outlier filtered box plot
transactions_df = transactions_df.loc[transactions_df["PURCHASE_AMOUNT"] < 3000]
plt.figure(figsize =(3, 3))
plt.boxplot(transactions_df["PURCHASE_AMOUNT"])
plt.show()

- Purchase quantity outlier

In [ ]:
# Purchase quantity box plot
plt.figure(figsize =(3, 3))
plt.boxplot(transactions_df["PURCHASE_QUANTITY"])
plt.show()

- Remove purchase_quantity > 400 and plot boxplot again.

In [ ]:
# Purchase quantity filtered box plot
transactions_df = transactions_df.loc[transactions_df["PURCHASE_QUANTITY"] < 400]
plt.figure(figsize =(3, 3))
plt.boxplot(transactions_df["PURCHASE_QUANTITY"])
plt.show()

- Filter purchase_amount & purchase_quantity from dataset. 
- Outliers are shown below:
    - Purchase amount > 3000
    - Purchase quantity > 400

In [ ]:
-- Remove outliers stated above
CREATE OR REPLACE TABLE store_2_preprocessed_transactions AS
SELECT * FROM store_2_preprocessed_transactions
WHERE 
    purchase_amount < 3000 AND
    purchase_quantity < 400;


- To test out z-score filtering model performance

In [ ]:
CREATE OR REPLACE TABLE store_2_z_score_preprocessed_transactions AS
SELECT date, store_chain_id, purchase_amount FROM preprocessed_transactions
WHERE store_chain_id = 2;

In [ ]:
from scipy import stats
import matplotlib.pyplot as plt

session = get_active_session()
session.use_database("ml")
session.use_schema("retail_store")

z_transactions_df = session.table("store_2_z_score_preprocessed_transactions").select("purchase_amount")
z_transactions_df = z_transactions_df.to_pandas()

z_transactions_df["z_score"] = stats.zscore(z_transactions_df["PURCHASE_AMOUNT"])
z_score_filtered_df = z_transactions_df[z_transactions_df["z_score"].abs() <= 4]

plt.figure(figsize =(3, 3))
plt.boxplot(filtered_data["PURCHASE_AMOUNT"])
plt.show()

In [ ]:
CREATE OR REPLACE TABLE store_2_z_score_preprocessed_transactions AS
WITH stats AS (
    SELECT 
        AVG(purchase_amount) AS mean_value,
        STDDEV(purchase_amount) AS stddev_value,
    FROM
        store_2_z_score_preprocessed_transactions
),
z_scores AS (
    SELECT
        date,
        store_chain_id,
        purchase_amount,
        (purchase_amount - stats.mean_value) / stats.stddev_value as z_score
    FROM 
        store_2_z_score_preprocessed_transactions,
        stats
)
SELECT 
    date,
    store_chain_id, 
    SUM(purchase_amount) AS total_sales
FROM 
    z_scores
WHERE 
    ABS(z_score) <= 3    
GROUP BY store_chain_id, date
ORDER BY store_chain_id, date; 

### 3. Group transactions by store chain id and date.
- Filter out transaction rows where date >= min(offer_date)
- Aggregate total purchase amount per day
- Aggregate total customers per day
- Aggregate total purchase amount per purchase quantity per day


In [ ]:
CREATE OR REPLACE TABLE store_2_preprocessed_transactions AS
WITH filtered_transactions AS (
    SELECT 
        store_chain_id,
        date,
        customer_id,
        purchase_quantity,
        purchase_amount, 
        offer_date
    FROM store_2_preprocessed_transactions
    WHERE
        (SELECT COUNT(offer_date) FROM store_preprocessed_transactions) = 0 OR
        date < (SELECT MIN(offer_date) FROM store_2_preprocessed_transactions)
)
SELECT 
    store_chain_id,
    date,
    SUM(purchase_amount) AS total_sales,
    COUNT(DISTINCT customer_id) AS total_customers,
    SUM(purchase_amount) / NULLIF(SUM(purchase_quantity), 0) AS purchase_amt_per_qty
FROM filtered_transactions
GROUP BY store_chain_id, date
ORDER BY store_chain_id, date;


### 4. Create a new flag feature to capture weekend vs weekday.

In [ ]:
ALTER TABLE store_2_preprocessed_transactions
ADD COLUMN IF NOT EXISTS flag_weekend BOOLEAN;

UPDATE store_2_preprocessed_transactions
SET flag_weekend =
    CASE 
        WHEN EXTRACT (DOW FROM date) IN (0,6)
        THEN TRUE
        ELSE FALSE
    END;

### 5. Impute 0 for missing dates

In [ ]:
CREATE OR REPLACE TABLE store_2_preprocessed_transactions AS
WITH date_range AS(
    SELECT 
        MIN(date) AS min_date,
        MAX(date) AS max_date,
    FROM store_2_preprocessed_transactions
),

-- Create date table
date_table AS (
    SELECT 
        min_date AS date,
        max_date 
    FROM date_range
    UNION ALL
    SELECT
        DATEADD(day, 1, date),
        max_date
    FROM date_table
    WHERE date_table.date < date_table.max_date
)

-- Left join with transactions table
SELECT 
    dt.date,
    COALESCE(s.store_chain_id, 2) AS store_chain_id,
    COALESCE(s.total_sales, 0) AS total_sales,
    COALESCE(s.total_customers, 0) AS total_customers,
    COALESCE(s.purchase_amt_per_qty, 0) AS purchase_amt_per_qty,
    COALESCE(s.flag_weekend, CASE WHEN EXTRACT(DOW FROM dt.DATE) IN (0, 6) THEN TRUE ELSE FALSE END) AS flag_weekend
FROM date_table dt
LEFT JOIN store_2_preprocessed_transactions s
    ON dt.date = s.date
ORDER BY s.date;

### 6. Remove outliers
- total customers per day
- purchase amount per quantity

In [ ]:
transactions_df = session.table("store_2_preprocessed_transactions").select("total_customers", "purchase_amt_per_qty")
transactions_df = transactions_df.to_pandas()

In [ ]:
plt.figure(figsize =(3, 3))
plt.boxplot(transactions_df["TOTAL_CUSTOMERS"])
plt.show()

In [ ]:
plt.figure(figsize =(3, 3))
plt.boxplot(transactions_df["PURCHASE_AMT_PER_QTY"])
plt.show()

### 7. Change flag_weekend into binary

In [ ]:
CREATE OR REPLACE TABLE store_2_preprocessed_transactions AS
SELECT 
    date,
    store_chain_id, 
    total_sales,
    total_customers,
    purchase_amt_per_qty,
    CASE 
        WHEN flag_weekend = TRUE THEN 1
        WHEN flag_weekend = FALSE THEN 0
    END AS flag_weekend
FROM store_2_preprocessed_transactions;

### 8. Check for any NULL values

In [ ]:
transactions_df = session.table("store_2_preprocessed_transactions").select("*")
transactions_df = transactions_df.to_pandas()

In [ ]:
value = transactions_df.isnull().values.any()
print(value)

### 9. Sort table based on date

In [ ]:
CREATE OR REPLACE TABLE store_2_preprocessed_transactions AS
SELECT * FROM store_2_preprocessed_transactions
ORDER BY date;

In [ ]:
SELECT * FROM store_2_preprocessed_transactions;

In [ ]:
session.close()

### 10. Summary
- store_2_preprocessed_transactions table contains
    1. DATE: DATE
    2. STORE_CHAIN_ID: NUMBER(38,0)
    3. TOTAL_SALES: FLOAT
    4. TOTAL_CUSTOMERS: NUMBER(18,0)
    5. PURCHASE_AMT_PER_QTY: FLOAT
    6. FLAG_WEEKEND: NUMBER(1,0)
